In [1]:
# pip install tensorflow==2.15.1 

In [2]:
import os
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense, LSTM, Dropout, TimeDistributed, Bidirectional, Concatenate, GlobalAveragePooling1D, AdditiveAttention
from tensorflow.keras.optimizers import Adam
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

# Last inn 'X_train_LSTM' fra en CSV-fil
X_df = pd.read_csv(url_data+'X_tensorflow.csv')
# Konverter hele DataFrame til et NumPy array
X = X_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
y_df = pd.read_csv(url_data+'y_liberal.csv')
# Konverter hele DataFrame til et NumPy array
y = y_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
embeddings_GloVe = pd.read_csv(url_data+'embeddings_glove.csv')
# Konverter hele DataFrame til et NumPy array
embeddings_GloVe = embeddings_GloVe.to_numpy()


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [4]:
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf

import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense, LSTM, Dropout, TimeDistributed, Bidirectional, Concatenate, GlobalAveragePooling1D, AdditiveAttention
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard
from datetime import datetime

class TextClassifierHyperModel:
    def __init__(self, input_shape, embeddings_GloVe, num_classes, parallel_blocks):
        self.input_shape = input_shape
        self.embeddings_GloVe = embeddings_GloVe
        self.num_classes = num_classes
        self.parallel_blocks = parallel_blocks

    def build(self):
        config = wandb.config
        sequence_input = Input(shape=(self.input_shape,), dtype='int32')
        embedded_sequences = Embedding(input_dim=self.embeddings_GloVe.shape[0],
                                       output_dim=self.embeddings_GloVe.shape[1],
                                       weights=[self.embeddings_GloVe],
                                       trainable=False)(sequence_input)

        conv_blocks = []
        lstm_blocks = []

        for _ in range(self.parallel_blocks):
            conv = Conv1D(
                filters=config.conv_filter_units,
                kernel_size=1,
                activation='relu',
                padding='same',
                strides=1)(embedded_sequences)
            conv_dense = TimeDistributed(Dense(config.dense_units, activation='relu'))(conv)
            conv_blocks.append(conv_dense)

            lstm = Bidirectional(LSTM(
                units=config.lstm_units,
                return_sequences=True,
                dropout=config.lstm_dropout_rate,
                recurrent_dropout=config.lstm_r_dropout_rate
            ))(conv_dense)
            lstm_blocks.append(lstm)

        combined = Concatenate()(conv_blocks + lstm_blocks)
        attention_layer = AdditiveAttention(use_scale=True)
        attention_output = attention_layer([combined, combined], return_attention_scores=False)
        context_vector = GlobalAveragePooling1D()(attention_output)

        dense = Dense(units=config.dense_units, activation='relu')(context_vector)
        dropout = Dropout(config.dropout_rate)(dense)
        outdata = Dense(self.num_classes, activation='sigmoid')(dropout)
        model = Model(inputs=sequence_input, outputs=outdata)
        optimizer = Adam(learning_rate=config.learning_rate)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        return model

# Initialize Weights & Biases
wandb.init(project="LSTM_CNN_Attention")

# Define the sweep configuration
sweep_config = {
    'method': 'random',  # or 'grid', 'random'
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'conv_filter_units': {
            'values': [30, 31, 32]
        },
        'dense_units': {
            'distribution': 'int_uniform',
            'min': 60,  # Minimum verdi
            'max': 160  # Maksimum verdi
        },
        'lstm_units': {
            'distribution': 'int_uniform',
            'min': 60,  # Minimum verdi
            'max': 160  # Maksimum verdi
        },
        'lstm_dropout_rate': {
            'min': 0.0,
            'max': 0.04
        },
        'lstm_r_dropout_rate': {
            'min': 0.01,
            'max': 0.06
        },
        'dense_units':      {
            'distribution': 'int_uniform',
            'min': 60,  # Minimum verdi
            'max': 160  # Maksimum verdi
        },
        'learning_rate': {
            'distribution': 'log_uniform',
            'min': -9.21,  # log(1e-4)
            'max': -4.61   # log(1e-2)
        },
        'dropout_rate': {
            'min': 0.0,
            'max': 0.1
        }
    }
}

input_length = 20
num_classes = 1
parallel_blocks = 2


# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="LSTM_CNN_Attention")

# Function to train the model
def train():
    # Initialize a new wandb run
    wandb.init(reinit=True)
    
    hypermodel = TextClassifierHyperModel(input_length, embeddings_GloVe, num_classes, parallel_blocks)
    model = hypermodel.build()
    
    # Train the model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks=[WandbCallback()])
    wandb.finish()

# Run the sweep
wandb.agent(sweep_id, train)

INFO:tensorflow:Assets written to: c:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\git_NLP_Notebooks\wandb\run-20240418_152212-rx6cfrzt\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\git_NLP_Notebooks\wandb\run-20240418_152212-rx6cfrzt\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\git_NLP_Notebooks\wandb\run-20240418_152212-rx6cfrzt\files\model-best)... Done. 0.1s


282/282 [==============================] - 77s 273ms/step - loss: 0.4514 - accuracy: 0.7892 - val_loss: 0.5206 - val_accuracy: 0.7510


accuracy,▁▄▆▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▅▆██
val_loss,█▄▂▂▁
accuracy,0.78915
best_epoch,4
best_val_loss,0.52065
epoch,4
loss,0.45145
val_accuracy,0.75104


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: xwt0ra3z with config:
wandb: 	conv_filter_units: 32
wandb: 	dense_units: 103
wandb: 	dropout_rate: 0.08109940785150405
wandb: 	learning_rate: 0.00014656523099950393
wandb: 	lstm_dropout_rate: 0.03197132465673359
wandb: 	lstm_r_dropout_rate: 0.01708628534588882
wandb: 	lstm_units: 92


Epoch 1/5
282/282 [==============================] - ETA: 0s - loss: 0.6412 - accuracy: 0.6474

c:\Users\bugat\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
best_model.save(r'C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model')

INFO:tensorflow:Assets written to: C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model\assets


INFO:tensorflow:Assets written to: C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model\assets


In [ ]:
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 20, 100)              2223500   ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 20, 31)               3131      ['embedding[0][0]']           
                                                                                                  
 conv1d_1 (Conv1D)           (None, 20, 31)               3131      ['embedding[0][0]']           
                                                                                              